In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from collections import OrderedDict
import pandas as pd
# Make the graphs a bit prettier, and bigger
pd.set_option('display.mpl_style', 'default')

# This is necessary to show lots of columns in pandas 0.12. 
# Not necessary in pandas 0.13.
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 20)

import numpy as np
import math
# The usual preamble
%matplotlib inline
%pylab inline
import matplotlib.pyplot as plt
plt.rcParams['axes.color_cycle'] = ['r', 'g', 'b', 'c']
plt.rcParams['lines.color'] = 'r'
plt.rcParams['figure.figsize'] = (15, 5)

import process
from modeling import categorize
import json

Populating the interactive namespace from numpy and matplotlib


/Users/along528/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import pickle
#clf = pickle.load(open('randomforestclassifier.p','rb'))
best_feature_names = pickle.load(open('best_features_names.p','rb'))
model = pickle.load(open('randomforestclassifier_fixed.p','rb'))
print model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=15, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [3]:
df_with_rpsi = process.add_features(process.get_data(munge=True,with_traffic=True))
#df_all = process.add_features(process.get_data(munge=False,with_traffic=False))
df_all = process.add_features(process.get_data(munge=True,with_traffic=False,drop_features=False))
df_all_drop = process.add_features(process.get_data(munge=True,with_traffic=False))

#df_all = df_all_munge.merge(df_all,how='left') #in case dropped some during munging
df_all = df_all.merge(pd.DataFrame(df_with_rpsi['rpsi']),how='left',right_index=True,left_index=True)
df_all['is_measured'] = np.isnan(df_all['rpsi'])==False


In [4]:
val_df,_ = process.get_split_add_data()
processor = process.Processor(val_df[best_feature_names])
X_predict,_ = processor.get_scaled_Xy(df_all_drop[best_feature_names])
y_predict = model.predict(X_predict)
df_all_predictions = pd.DataFrame(y_predict,columns=['rpsi_predicted'],index=df_all_drop.index)
df_all = df_all.merge(df_all_predictions,how='left',right_index=True,left_index=True)
df_all = df_all.fillna(-1)
df_all = df_all[((df_all['rpsi']==-1) & (df_all['rpsi_predicted']==-1))==False]
df_all

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,sgtmin_per_capita,sgtmax_per_capita,entrymin_per_capita,entrymax_per_capita,nummrkcars_per_capita,numothmrk_per_capita,numumkcars_per_capita,numothunm_per_capita,numplanes_per_capita,numcopters_per_capita,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,Anchorage Police Department,Anchorage,AK,99507,3,399,385,0,174,0,559,0,0,0,0,0,6,0,12,0,6,0,7,0,182,6,18,182,76,0,...,0.318133,0.380107,0.227238,0.392502,0.000698,0.000112,0.000430,0.000079,0,0.000000,0.000000,0.000050,0.000062,0.000000,0.000000,0.092703,0.087302,0.131329,0.228111,1.000000,1.000000,1.000000,0.252708,0.148649,0.940382,1.345001,0.699168,-1,False,2
1001,Ketchikan Police Department,Ketchikan,AK,99901,3,23,23,0,10,1,33,1,0,0,0,0,0,0,0,0,0,0,0,0,23,0,2,0,0,0,...,4.186778,5.464836,3.378368,4.408499,0.000592,0.000296,0.000296,0.000000,0,0.000000,0.000000,0.000000,0.001036,0.000666,0.000000,0.008583,0.005714,0.070175,0.000000,1.000000,1.000000,0.000000,1.000000,0.214286,0.000000,1.148574,0.000000,-1,False,2
16,Tuscaloosa Police Department,Tuscaloosa,AL,35401,3,269,264,0,75,0,339,0,0,0,0,0,0,0,11,0,1,0,0,0,157,7,10,157,58,0,...,0.390976,0.518801,0.341532,0.453193,0.001030,0.000160,0.000630,0.000115,0,0.000027,0.000036,0.000071,0.000337,0.000426,0.000009,0.722283,0.688996,1.017279,1.487805,1.000000,1.000000,1.000000,1.487805,0.783784,0.369151,1.517748,0.243223,-1,False,1
13,Mobile Police Department,Mobile,AL,36606,3,532,526,0,254,96,780,96,0,6,0,0,0,0,5,0,1,0,0,0,304,30,5,345,97,0,...,0.125447,0.194642,0.098319,0.152487,0.000875,0.000096,0.000488,0.000137,0,0.000000,0.000004,0.000059,0.000369,0.000148,0.000000,0.887893,0.763822,0.922970,1.873085,1.000000,1.000000,1.000000,1.873085,2.195122,0.622467,1.391109,0.447461,-1,False,0
1004,Anniston Police Department,Anniston,AL,36202,3,92,81,0,33,0,114,0,0,0,0,0,0,0,1,0,0,0,0,0,69,0,0,0,0,0,...,0.482391,0.733924,0.413878,0.620859,0.000946,0.000056,0.000226,0.000056,0,0.000000,0.000000,0.000014,0.000565,0.000085,0.000000,0.392830,0.413985,0.371879,0.723881,1.000000,1.000000,1.000000,0.723881,0.551020,0.230880,1.360448,0.169709,-1,False,0
5,Birmingham Police Department,Birmingham,AL,35203,3,914,768,0,400,0,1168,0,0,0,0,0,4,0,0,0,1,0,0,0,462,462,12,528,137,1,...,0.100501,0.110821,0.061666,0.095531,0.000701,0.000077,0.000389,0.000109,0,0.000000,0.000000,0.000047,0.000132,0.000000,0.000000,0.872831,0.813041,0.856183,1.704591,1.000000,1.000000,0.618497,2.277439,2.373333,1.421503,0.735633,1.932355,-1,False,1
15,Montgomery Police Department,Montgomery,AL,36101,3,510,491,0,97,0,588,0,0,0,0,0,2,0,1,0,2,0,0,0,212,12,15,212,88,2,...,0.192620,0.274193,0.149108,0.212261,0.001006,0.000158,0.000619,0.000116,0,0.000000,0.000000,0.000070,0.000415,0.000000,0.000061,1.488363,1.322901,1.429854,1.529372,5.372549,1.235294,1.173724,4.019802,1.941748,0.723585,1.518510,0.476510,-1,False,2
7,Dothan Police Department,Dothan,AL,36303,3,157,148,0,100,0,248,0,0,0,0,15,0,0,0,0,0,0,0,0,80,0,12,80,38,22,...,0.479825,0.733411,0.382866,0.586729,0.001243,0.000112,0.000497

In [5]:
df_all['rpsi_final'] =  df_all['rpsi_predicted']
df_all['rpsi_final']
for index,series in df_all.iterrows():
    if series['is_measured']:
        df_all.loc[index,'rpsi_final'] = categorize(series['rpsi'])
df_all[df_all['is_measured']]

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,sgtmax_per_capita,entrymin_per_capita,entrymax_per_capita,nummrkcars_per_capita,numothmrk_per_capita,numumkcars_per_capita,numothunm_per_capita,numplanes_per_capita,numcopters_per_capita,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_final
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1212,New London Police Department,New London,CT,6320,3,86,78,0,0,0,78,0,0,0,0,0,0,0,2,0,0,0,0,0,51,2,2,0,0,0,...,2.166419,1.612956,1.939023,0.001086,0.000181,0.000652,0.000145,0,0,0.000036,0.000109,0.000435,0.000579,0.000000,0.288592,0.299111,0.126984,1.000000,1,1.000000,1.000000,1.000000,1.000000,0.588391,1.315987,0.447110,1.322864,True,1,1
1222,Westport Police Department,Westport,CT,6880,3,65,65,15,13,36,78,51,0,15,0,36,0,0,0,0,1,0,0,0,37,0,0,0,0,0,...,2.965287,1.670695,2.456337,0.000675,0.000150,0.000450,0.000150,0,0,0.000075,0.000038,0.000075,0.000525,0.000000,0.013104,0.003862,0.256944,1.000000,1,1.000000,1.000000,1.000000,1.200000,0.000000,1.031163,0.000000,3.067020,True,1,2
184,Waterbury Police Department,Waterbury,CT,6702,3,300,300,0,53,11,353,11,0,0,0,0,0,0,1,0,0,0,0,0,168,12,7,185,58,0,...,0.587504,0.406117,0.414127,0.001196,0.000190,0.000734,0.000136,0,0,0.000000,0.000082,0.000000,0.000027,0.000027,0.341569,0.290306,0.242741,0.987952,1,1.000000,1.000000,1.000000,0.565217,0.348868,1.361861,0.256170,1.356672,True,1,1
175,Hartford Police Department,Hartford,CT,6120,3,592,417,0,92,97,509,97,0,0,0,0,2,0,3,0,1,0,0,0,150,64,8,250,65,15,...,0.579822,0.331822,0.499568,0.001477,0.000234,0.000912,0.000169,0,0,0.000024,0.000105,0.000347,0.000016,0.000000,1.314375,1.031085,0.665044,1.354497,1,1.000000,1.000000,1.411003,1.000000,0.389109,2.118801,0.183646,1.708121,True,1,1
1223,Wethersfield Police Department,Wethersfield,CT,6109,3,47,47,0,11,23,58,23,0,0,0,0,0,0,1,0,0,0,0,0,35,3,1,0,0,0,...,3.705319,2.194957,2.443738,0.000487,0.000075,0.000262,0.000075,0,0,0.000037,0.000075,0.000000,0.000037,0.000000,0.035182,0.043460,0.081871,1.000000,1,1.000000,1.000000,1.000000,0.000000,1.350963,0.950585,1.421190,1.314888,True,1,1
1214,Plainfield Police Department,Plainfield,CT,6374,3,19,18,0,4,4,22,4,0,0,0,0,0,0,0,0,0,0,0,0,14,0,1,0,0,0,...,6.329870,5.587006,6.324137,0.000597,0.000239,0.000478,0.000000,0,0,0.000000,0.000000,0.000597,0.000000,0.000000,0.010043,0.006611,0.006897,1.000000,1,1.000000,1.000000,1.000000,1.000000,0.000000,0.827909,0.000000,3.972093,True,0,2
170,Danbury Police Department,Danbury,CT,6810,3,152,148,0,6,0,154,0,0,9,0,0,0,0,5,0,0,1,0,0,90,0,5,90,22,0,...,0.856433,0.596734,0.765881,0.000519,0.000111,0.000334,0.000074,0,0,0.000012,0.000000,0.000000,0.000000,0.000000,0.105144,0.083884,0.373494,0.637363,1,0.637363,1.000000,1.000000,0.411765,0.376702,1.307061,0.288205,1.210668,True,1,0
174,Hamden Police Department,Hamden,CT,6518,3,107,101,0,25,25,126,25,0,1,0,0,1,0,1,0,0,0,0,0,60,3,2,62,28,0,...,1.170128,0.796690,1.060739,0.000377,0.000098,0.000328,0.000066,0,0,0.000000,0.000066,0.000197,0.000000,0.000016,0.294419,0.269127,0.146006,1.000000,1,1.000000,1.000000,1.000000,0.651163,0.147321,1.35

In [6]:
df_all.to_sql('app_db',process.engine,if_exists='replace')

In [7]:
df_all.to_csv('app_db.csv')

In [8]:
df_all[df_all['agency'].str.contains('raleigh'.title())]

,agency,city,state,zipcode,agcytype,swnauthemp,swnftemp,swnptemp,civftemp,civptemp,totftemp,totptemp,ftreserveswn,ptreserveswn,ftreserveciv,ptreserveciv,ftgangoff,ptgangoff,ftdrugoff,ptdrugoff,ftterroff,pterroff,fthumtrfoff,pthumtrfoff,numrespoff,numcpo,numsro,numpatr,numinvst,numjail,...,sgtmax_per_capita,entrymin_per_capita,entrymax_per_capita,nummrkcars_per_capita,numothmrk_per_capita,numumkcars_per_capita,numothunm_per_capita,numplanes_per_capita,numcopters_per_capita,numboats_per_capita,nummotor_per_capita,numcarcam_per_capita,numfixcam_per_capita,nummobcam_per_capita,black_over_white_population_disparity,black_over_white_income_disparity,black_over_white_institutionalized_disparity,black_over_white_institutionalized_adult_disparity,black_over_white_institutionalized_adult_federal_detention_disparity,black_over_white_institutionalized_adult_federal_prison_disparity,black_over_white_institutionalized_adult_state_prison_disparity,black_over_white_institutionalized_adult_local_jail_disparity,black_over_white_institutionalized_juvenile_disparity,black_officer_disparity,white_officer_disparity,black_over_white_officer_disparity,rpsi,is_measured,rpsi_predicted,rpsi_final
surveyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
594,Raleigh Police Dept,Raleigh,NC,27602,3,764,710,0,114,0,824,0,0,0,0,0,1,0,1,0,2,0,0,0,305,6,10,305,110,0,...,0.148188,0.066697,0.113468,0.000654,0.000074,0.000364,0.000102,0,0,0,0.000043,0.000143,0,0,0.422356,0.378851,0.985024,1.365526,1,1,1.14524,1.757396,3.761905,0.547183,1.317992,0.415164,1.726268,True,1,1


## split by quantiles for plotting

In [9]:
featuremap = OrderedDict()
featuremap['entrymin'] = ['Minimum Salary','USD']
featuremap['opbudget_per_capita'] = ['Budget per capita','USD/person']
featuremap['urban_per_capita'] = ['Urban residents per capita','']
featuremap['population_black_per_capita'] = ['Black population per capita','']
featuremap['chiefmin'] = ['Minimum Chief Officer Salary','']
featuremap['entrymax'] = ['Maximum Officer Salary','']
featuremap['total_income_estimate_all_per_capita'] = ['Total Income per capita','']
featuremap['black_over_white_institutionalized_adult_local_jail_disparity'] =  ['Black/White population in local jail','']
featuremap['total_income_estimate_white_per_capita'] = ['Total Income for white population per capita','']
featuremap['nummrkcars_per_capita'] = ['Number of marked cars per capita','']
featuremap['swnftemp_per_capita'] = ['Number of officers with arrest power per capita','']
featuremap['institutionalized_all_per_capita'] = ['Institutionalized residents per capita','']
featuremap['totacad'] = ['Training','Hours']
featuremap['black_over_white_population_disparity'] = ['Black/White population disparity','']




In [20]:
splitby = [.20,.40,.60,0.80,0.95]
quantiles = []
quantiles.append(df_all.quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==0].quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==1].quantile(splitby))
quantiles.append(df_all[df_all['rpsi_final']==2].quantile(splitby))
quantiles_df = pd.concat(quantiles)
#.to_json('test.json')

In [21]:
def build_bullet_json(quantiles_df, all_df,surveyid=348,category=0):
    json_output = []
    indices_all = [0,1,2,3,4]
    indices_cat = []
    for i in indices_all:
        indices_cat.append(i+(category+1)*len(indices_all))
    for index,series in quantiles_df.iteritems():
        #assumes 4 quantiles
        if index not in featuremap: continue
        title = featuremap[index][0]
        subtitle = featuremap[index][1]
        quantiles_all = []
        for i in indices_all:
            quantiles_all.append(series.iloc[i])
        quantiles_cat = []
        for i in indices_cat:
            quantiles_cat.append(series.iloc[i])
        json_output.append({})
        json_output[-1]['title'] = title
        json_output[-1]['subtitle'] = subtitle
        json_output[-1]['ranges'] = quantiles_all
        json_output[-1]['measures'] = quantiles_cat
        json_output[-1]['markers'] = all_df[all_df.index==surveyid][index].tolist()
    return json_output

        

In [22]:
for index,series in df_all.iterrows():
    category = int(series['rpsi_final'])
    surveyid = index
    output = build_bullet_json(quantiles_df,df_all,
                               surveyid=surveyid,category=category)
    output = json.dumps(output).replace('\\"','"').\
        replace('"[','[').replace(']"',']')
    with open('json/'+str(surveyid)+'.json', 'w') as outfile:
        outfile.write(output)
    